In [3]:
import pymysql
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scikit_posthocs as sp
import scipy.stats as stats
import csv
import matplotlib.colors as mcolors
import os

from statsmodels.stats.multicomp import MultiComparison

In [4]:
con = pymysql.connect(host="localhost", user="gabic", passwd="1234", db="doenca_renal_cronica")
con.select_db('doenca_renal_cronica')
cursor = con.cursor()  
cursor.connection.autocommit(True)
# csv
#esse pacientes_tfg será para inserir os id ds pacientes que tem  tfg, separar por exame e pegar o valor recalculado 
# pacientes_tfg= pd.read_csv('calculo_tfg_correto.csv')
# pacientes_tfg = pacientes_tfg.replace(np.nan, '0')

In [5]:
dados_consulta = pd.read_sql('SELECT * FROM doenca_renal_cronica.dados_consulta',con=con)
cidade = pd.read_sql('SELECT * FROM doenca_renal_cronica.cidade',con=con)
ubs = pd.read_sql('SELECT * FROM doenca_renal_cronica.ubs',con=con)
aval_funcao_renal = pd.read_sql('SELECT * FROM doenca_renal_cronica.aval_funcao_renal',con=con)
consulta_has = pd.read_sql('SELECT * FROM doenca_renal_cronica.consulta_has',con=con)
consulta_drc = pd.read_sql('SELECT * FROM doenca_renal_cronica.consulta_drc',con=con)
consulta_dm = pd.read_sql('SELECT * FROM doenca_renal_cronica.consulta_dm',con=con)
# calculo_tfg = pd.read_sql('SELECT * FROM doenca_renal_cronica.calculo_tfg',con=con)
calculo_tfg = pd.read_csv('calculo_tfg_correto.csv')
data = pd.read_csv('banco/Dados2010-2014DesfechoFasesETC2.csv')
idade = pd.DataFrame(data['Idade'])

preparacao_trs = pd.read_sql('SELECT * FROM doenca_renal_cronica.preparacao_trs',con=con)
raca = pd.read_sql('SELECT * FROM doenca_renal_cronica.raca',con=con)
instrucao = pd.read_sql('SELECT * FROM doenca_renal_cronica.instrucao',con=con)
paciente = pd.read_sql('SELECT * FROM doenca_renal_cronica.paciente',con=con)
exame_fisico = pd.read_sql('SELECT * FROM doenca_renal_cronica.exame_fisico',con=con)
avaliacao_tratamento = pd.read_sql('SELECT * FROM doenca_renal_cronica.avaliacao_tratamento',con=con)
medicamento = pd.read_sql('SELECT * FROM doenca_renal_cronica.medicamento',con=con)
complicacao_funcao_renal = pd.read_sql('SELECT * FROM doenca_renal_cronica.complicacao_funcao_renal',con=con)

#dicionario com todas tabelas
# dic = {'dados_consulta':dados_consulta,'cidade':cidade,'ubs':ubs,'aval_funcao_renal':aval_funcao_renal,
#        'consulta_has':consulta_has,'consulta_drc':consulta_drc,'consulta_dm':consulta_dm,'calculo_tfg':calculo_tfg,
#        'preparacao_trs':preparacao_trs,'raca':raca,'instrucao':instrucao,'paciente':paciente,'exame_fisico':exame_fisico,
#        'avaliacao_tratamento':avaliacao_tratamento,'medicamento':medicamento,'complicacao_funcao_renal':complicacao_funcao_renal
#       }
dic = {'dados_consulta':dados_consulta,'aval_funcao_renal':aval_funcao_renal,'raca':raca, 'idade':idade,
       'calculo_tfg':calculo_tfg,'preparacao_trs':preparacao_trs,'paciente':paciente,'exame_fisico':exame_fisico,
       'avaliacao_tratamento':avaliacao_tratamento,'medicamento':medicamento,'complicacao_funcao_renal':complicacao_funcao_renal
      }
dic['paciente']['sexo'] = dic['paciente']['sexo'].replace(['Feminino','Masculino'],[1,2])
dic['exame_fisico']['tabagismo'] = dic['exame_fisico']['tabagismo'].replace(['Ex','Sim'],[0,1])
dic['paciente']=dic['paciente'].drop(columns = ['renda_sm','renda_familiar_sm'])
dic['paciente']=dic['paciente'].drop(columns = ['tamanho_familia'])
dic['raca']=dic['raca'].replace(['Branca','Parda','Preta','Indigena','Amarela'],[1,2,3,4,5])


#passand str para  int ds estagios
dic['dados_consulta']['estagioI']= dic['dados_consulta']['estagioI'].replace('1',1)
dic['dados_consulta']['estagioI']= dic['dados_consulta']['estagioI'].replace('2',2)
dic['dados_consulta']['estagioI']= dic['dados_consulta']['estagioI'].replace('3a',31)
dic['dados_consulta']['estagioI']= dic['dados_consulta']['estagioI'].replace('3b',32)
dic['dados_consulta']['estagioI']= dic['dados_consulta']['estagioI'].replace('4',4)
dic['dados_consulta']['estagioI']= dic['dados_consulta']['estagioI'].replace('5',5)

dic['dados_consulta']['estagioF']= dic['dados_consulta']['estagioF'].replace('1',1)
dic['dados_consulta']['estagioF']= dic['dados_consulta']['estagioF'].replace('2',2)
dic['dados_consulta']['estagioF']= dic['dados_consulta']['estagioF'].replace('3a',31)
dic['dados_consulta']['estagioF']= dic['dados_consulta']['estagioF'].replace('3b',32)
dic['dados_consulta']['estagioF']= dic['dados_consulta']['estagioF'].replace('4',4)
dic['dados_consulta']['estagioF']= dic['dados_consulta']['estagioF'].replace('5',5)

/home/gabic9814/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (214,222,223,229) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
#substituindo os valores para Nan 
for tabela in dic:
    for coluna in dic[tabela].keys():
        dic[tabela][coluna]=dic[tabela][coluna].replace([0,'0','0.0',''], np.nan)

In [7]:
#retirando os ids, os valores de exames totais, data de aniversário, data de inicio e de final de tratamento
for tabela in dic:
    for coluna in dic[tabela].keys():
        i_split = coluna.split('_')
        if i_split[0]=='id'and i_split[1]!='original':
            dic[tabela]=dic[tabela].drop(columns=[coluna], errors='ignore')
#         if i_split[0]=='dt':
#             dic[tabela]=dic[tabela].drop(columns=[coluna], errors='ignore')
        if i_split[0]=='total':
            dic[tabela]=dic[tabela].drop(columns=[coluna], errors='ignore')
        if i_split[0]=='data':
            dic[tabela]=dic[tabela].drop(columns=[coluna], errors='ignore')

In [8]:
#df é um dataframe auxiliar que ajuda a transformar o dicionario em um só para ficar mais fácil de separar 
#os pacientes com pelo menos um valor de tfg 
df = pd.DataFrame(dic['paciente'])
for tabela in dic.keys():
    for coluna in dic[tabela].keys():
        if tabela!='paciente':
            df[coluna] =dic[tabela][coluna]

In [14]:
type(df['estagioI'][1])

int

In [9]:
# print(str(df['tfg_1_2011'].replace([0,'0','0.0',''],np.nan).describe()['count']))#1283
# print(str(df['tfg_2_2011'].replace([0,'0','0.0',''],np.nan).describe()['count']))#maior com 1604
# print(str(df['tfg_1_2012'].replace([0,'0','0.0',''],np.nan).describe()['count']))#1694
# print(str(df['tfg_2_2012'].replace([0,'0','0.0',''],np.nan).describe()['count']))#maior com 1724   
# print(str(df['tfg_1_2013'].replace([0,'0','0.0',''],np.nan).describe()['count']))#1961
# print(str(df['tfg_2_2013'].replace([0,'0','0.0',''],np.nan).describe()['count']))#maior com 2093   
# print(str(df['tfg_1_2014'].replace([0,'0','0.0',''],np.nan).describe()['count']))#maior 2310
# print(str(df['tfg_2_2014'].replace([0,'0','0.0',''],np.nan).describe()['count']))#2083   

In [10]:
#Criar um data frame para cada ano de medida do tfg

# index_2011 = [i for i in range(int(df['tfg_2_2011'].replace([0,'0','0.0',''],np.nan).describe()['count']))]
# tfg_2011 = pd.DataFrame(columns=df.keys(), index=np.array(index_2011))
# id_ultimo_2011=0

# index_2012 = [i for i in range(int(df['tfg_2_2012'].replace([0,'0','0.0',''],np.nan).describe()['count']))]
# tfg_2012 = pd.DataFrame(columns=df.keys(), index=np.array(index_2012))
# id_ultimo_2012=0

# index_2013 = [i for i in range(int(df['tfg_2_2013'].replace([0,'0','0.0',''],np.nan).describe()['count']))]
# tfg_2013 = pd.DataFrame(columns=df.keys(), index=np.array(index_2013))
# id_ultimo_2013=0

# index_2014 = [i for i in range(int(df['tfg_1_2014'].replace([0,'0','0.0',''],np.nan).describe()['count']))]
# tfg_2014 = pd.DataFrame(columns=df.keys(), index=np.array(index_2014))
# id_ultimo_2014=0

# for id_tfg in range(len(dic['paciente']['id_original'])):
    
#     if df['tfg_1_2011'][id_tfg]!=0 or df['tfg_2_2011'][id_tfg]!=0:
#         tfg_2011.loc[id_ultimo_2011]=df.loc[id_tfg]
#         id_ultimo_2011+=1
#     if df['tfg_1_2012'][id_tfg]!=0 or df['tfg_2_2012'][id_tfg]!=0:
#         tfg_2012.loc[id_ultimo_2012]=df.loc[id_tfg]
#         id_ultimo_2012+=1
#     if df['tfg_1_2013'][id_tfg]!=0 or df['tfg_2_2013'][id_tfg]!=0:
#         tfg_2013.loc[id_ultimo_2013]=df.loc[id_tfg]
#         id_ultimo_2013+=1
#     if df['tfg_1_2014'][id_tfg]!=0 or df['tfg_2_2014'][id_tfg]!=0:
#         tfg_2014.loc[id_ultimo_2014]=df.loc[id_tfg]
#         id_ultimo_2014+=1
  

# tfg_2011 = tfg_2011.replace([0,'0','0.0',''], np.nan)
# tfg_2011.drop(columns=['id_original'], inplace=True)
# tfg_2011 = tfg_2011.select_dtypes(include = ['float64', 'int64'])

# tfg_2012 = tfg_2012.replace([0,'0','0.0',''], np.nan)
# tfg_2012.drop(columns=['id_original'], inplace=True)
# tfg_2012 = tfg_2012.select_dtypes(include = ['float64', 'int64'])

# tfg_2013 = tfg_2013.replace([0,'0','0.0',''], np.nan)
# tfg_2013.drop(columns=['id_original'], inplace=True)
# tfg_2013 = tfg_2013.select_dtypes(include = ['float64', 'int64'])

# tfg_2014 = tfg_2014.replace([0,'0','0.0',''], np.nan)
# tfg_2014.drop(columns=['id_original'], inplace=True)
# tfg_2014 = tfg_2014.select_dtypes(include = ['float64', 'int64'])


In [15]:
# tfg_2011.to_csv('tfg_2011.csv')
# tfg_2012.to_csv('tfg_2012.csv')
# tfg_2013.to_csv('tfg_2013.csv')
# tfg_2014.to_csv('tfg_2014.csv')

In [61]:
tfg_2011=pd.read_csv('tfg_2011.csv')
tfg_2012=pd.read_csv('tfg_2012.csv')
tfg_2013=pd.read_csv('tfg_2013.csv')
tfg_2014=pd.read_csv('tfg_2014.csv')

In [62]:
#correlação só com as colunas de tfg
index = [i for i in range(len(tfg_2011.keys()))]
head1 = ['tfg_1_2011', 'tfg_2_2011']
head2 = ['tfg_1_2012','tfg_2_2012']
head3 = ['tfg_1_2013','tfg_2_2013']
head4 = ['tfg_1_2014','tfg_2_2014']
corr_tfg_2011 = pd.DataFrame(columns=head1, index=index)
corr_tfg_2012 = pd.DataFrame(columns=head2, index=index)
corr_tfg_2013 = pd.DataFrame(columns=head3, index=index)
corr_tfg_2014 = pd.DataFrame(columns=head4, index=index)
corr1= []
corr2= []
corr3= []
corr4= []

for coluna in range(len(tfg_2011.keys())):
    
    corr1.append(tfg_2011[tfg_2011.keys()[coluna]].corr(tfg_2011[head1[0]]))
    corr1.append(tfg_2011[tfg_2011.keys()[coluna]].corr(tfg_2011[head1[1]]))
    
    corr2.append(tfg_2012[tfg_2012.keys()[coluna]].corr(tfg_2012[head2[0]]))
    corr2.append(tfg_2012[tfg_2012.keys()[coluna]].corr(tfg_2012[head2[1]]))
    
    corr3.append(tfg_2013[tfg_2013.keys()[coluna]].corr(tfg_2013[head3[0]]))
    corr3.append(tfg_2013[tfg_2013.keys()[coluna]].corr(tfg_2013[head3[1]]))
    
    corr4.append(tfg_2014[tfg_2014.keys()[coluna]].corr(tfg_2014[head4[0]]))
    corr4.append(tfg_2014[tfg_2014.keys()[coluna]].corr(tfg_2014[head4[1]]))
    
    corr_tfg_2011.loc[coluna] = corr1
    corr_tfg_2012.loc[coluna] = corr2
    corr_tfg_2013.loc[coluna] = corr3
    corr_tfg_2014.loc[coluna] = corr4
    corr1= []
    corr2= []
    corr3= []
    corr4= []
   

new = tfg_2011.keys()
new = [*new]
old = [i for i in range(len(tfg_2011.keys()))]
# columns = {old[i]:new[i] for i in range(len(df.keys()))}
index = {old[i]:new[i] for i in range(len(tfg_2011.keys()))}
corr_tfg_2011=corr_tfg_2011.rename(index)
corr_tfg_2012=corr_tfg_2012.rename(index)
corr_tfg_2013=corr_tfg_2013.rename(index)
corr_tfg_2014=corr_tfg_2014.rename(index)


corr_tfg_2011=corr_tfg_2011.replace(np.nan,0.0)
corr_tfg_2012=corr_tfg_2012.replace(np.nan,0.0)
corr_tfg_2013=corr_tfg_2013.replace(np.nan,0.0)
corr_tfg_2014=corr_tfg_2014.replace(np.nan,0.0)


indexNames = corr_tfg_2011[ corr_tfg_2011['tfg_1_2011'] == 0.0 ].index
corr_tfg_2011.drop(indexNames , inplace=True)
corr_tfg_2012.drop(indexNames , inplace=True)
corr_tfg_2013.drop(indexNames , inplace=True)
corr_tfg_2014.drop(indexNames , inplace=True)
tfg_2011.drop(columns=indexNames,inplace=True)
tfg_2012.drop(columns=indexNames,inplace=True)
tfg_2013.drop(columns=indexNames,inplace=True)
tfg_2014.drop(columns=indexNames,inplace=True)

In [65]:
corr_tfg_2011.style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'),axis=0)

,tfg_1_2011,tfg_2_2011
Unnamed: 0,0.13,0.11
sexo,-0.022,0.017
tempoAcomp,-0.058,-0.082
estagioI_alb,-0.76,-0.56
estagioF_alb,-0.44,-0.35
creatinina_inicial,-0.68,-0.61
creatinina_final,-0.54,-0.51
albumina_inicial,0.23,0.1
albumina_final,-0.18,0.23
rel_alb_creat_uauc_inicial,-0.055,-0.0082


In [69]:
corr_tfg_2012.style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'),axis=0)

,tfg_1_2012,tfg_2_2012
Unnamed: 0,0.16,0.12
sexo,-0.012,-0.042
tempoAcomp,-0.12,-0.1
estagioI_alb,-0.43,-0.48
estagioF_alb,-0.45,-0.51
creatinina_inicial,-0.64,-0.61
creatinina_final,-0.54,-0.53
albumina_inicial,0.19,-0.11
albumina_final,0.87,0.57
rel_alb_creat_uauc_inicial,-0.055,-0.054


In [70]:
corr_tfg_2013.style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'),axis=0)

,tfg_1_2013,tfg_2_2013
Unnamed: 0,0.088,0.14
sexo,0.044,-0.019
tempoAcomp,-0.064,-0.14
estagioI_alb,-0.46,-0.33
estagioF_alb,-0.39,-0.53
creatinina_inicial,-0.27,-0.45
creatinina_final,-0.29,-0.5
albumina_inicial,0.1,0.05
albumina_final,0.21,0.72
rel_alb_creat_uauc_inicial,-0.063,-0.035


In [71]:
corr_tfg_2014.style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'),axis=0)

,tfg_1_2014,tfg_2_2014
Unnamed: 0,0.17,0.076
sexo,-0.0062,-0.027
tempoAcomp,-0.15,-0.074
estagioI_alb,-0.42,-0.4
estagioF_alb,-0.51,-0.86
creatinina_inicial,-0.58,-0.19
creatinina_final,-0.55,-0.18
albumina_inicial,0.18,0.13
albumina_final,0.27,0.87
rel_alb_creat_uauc_inicial,-0.039,-0.012


In [68]:
corr_tfg_2011.style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'),axis=0).to_excel("corr_tfg_2011.xlsx")
corr_tfg_2012.style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'),axis=0).to_excel("corr_tfg_2012.xlsx")
corr_tfg_2013.style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'),axis=0).to_excel("corr_tfg_2013.xlsx")
corr_tfg_2014.style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'),axis=0).to_excel("corr_tfg_2014.xlsx")

In [75]:
# imprimir as variáveis que possuem valores de correlação maior que 0.5
##escreve o arquivo descrição afr
# with open ('descricao_afr.txt','w') as arquivo:
#     for ind in range(len(nomes_exames)):
#         arquivo.write('Nome:'+ nomes_exames[ind]+'\n')
#         arquivo.write(str(df_afr.loc[df_afr['exame'] == nomes_exames[ind], 'valor'].describe()))       
#         arquivo.write('\n')
with open ('var_corr_tfg_descrica.txt','w') as arquivo:
    for linha in range(len(corr_tfg_2011)):
        if tfg_2011.keys()[linha]!= 'tfg_1_2011' and tfg_2011.keys()[linha]!= 'tfg_2_2011' and tfg_2011.keys()[linha]!= 'tfg_1_2012' and tfg_2011.keys()[linha]!= 'tfg_2_2012' and tfg_2011.keys()[linha]!= 'tfg_1_2013' and tfg_2011.keys()[linha]!= 'tfg_2_2013' and tfg_2011.keys()[linha]!= 'tfg_1_2014' and tfg_2011.keys()[linha]!= 'tfg_2_2014':
            if corr_tfg_2011['tfg_1_2011'][linha]>=0.5 or corr_tfg_2011['tfg_2_2011'][linha]>=0.5:
                arquivo.write('valor da correlacao:'+'\n')
                arquivo.write('tfg_1_2011:'+str(corr_tfg_2011['tfg_1_2011'][linha])+'  ')
                arquivo.write('tfg_2_2011:'+str(corr_tfg_2011['tfg_2_2011'][linha])+'\n')
                arquivo.write(str(tfg_2011[tfg_2011.keys()[linha]].describe()))
                arquivo.write('\n')
                arquivo.write('\n')
            elif corr_tfg_2011['tfg_1_2011'][linha]<=-0.5 or corr_tfg_2011['tfg_2_2011'][linha]<=-0.5:
                arquivo.write('valor da correlacao:'+'\n')
                arquivo.write('tfg_1_2011:'+str(corr_tfg_2011['tfg_1_2011'][linha])+'  ')
                arquivo.write('tfg_2_2011:'+str(corr_tfg_2011['tfg_2_2011'][linha])+'\n')
                arquivo.write(str(tfg_2011[tfg_2011.keys()[linha]].describe()))
                arquivo.write('\n')
                arquivo.write('\n')

            elif corr_tfg_2012['tfg_1_2012'][linha]>=0.5 or corr_tfg_2012['tfg_2_2012'][linha]>=0.5:
                arquivo.write('valor da correlacao:'+'\n')
                arquivo.write('tfg_1_2012:'+str(corr_tfg_2012['tfg_1_2012'][linha])+'   ')
                arquivo.write('tfg_2_2012:'+str(corr_tfg_2012['tfg_2_2012'][linha])+'\n')
                arquivo.write(str(tfg_2012[tfg_2012.keys()[linha]].describe()))
                arquivo.write('\n')
                arquivo.write('\n')
            elif corr_tfg_2012['tfg_1_2012'][linha]<=-0.5 or corr_tfg_2012['tfg_2_2012'][linha]<=-0.5:
                arquivo.write('valor da correlacao:'+'\n')
                arquivo.write('tfg_1_2012:'+str(corr_tfg_2012['tfg_1_2012'][linha])+'   ')
                arquivo.write('tfg_2_2012:'+str(corr_tfg_2012['tfg_2_2012'][linha])+'\n')
                arquivo.write(str(tfg_2012[tfg_2012.keys()[linha]].describe()))
                arquivo.write('\n')
                arquivo.write('\n')

            elif corr_tfg_2013['tfg_1_2013'][linha]>=0.5 or corr_tfg_2013['tfg_2_2013'][linha]>=0.5:
                arquivo.write('valor da correlacao:'+'\n')
                arquivo.write('tfg_1_2013:'+str(corr_tfg_2013['tfg_1_2013'][linha])+'   ')
                arquivo.write('tfg_2_2013:'+str(corr_tfg_2013['tfg_2_2013'][linha])+'\n')
                arquivo.write(str(tfg_2013[tfg_2013.keys()[linha]].describe()))
                arquivo.write('\n')
                arquivo.write('\n')
            elif corr_tfg_2013['tfg_1_2013'][linha]<=-0.5 or corr_tfg_2013['tfg_2_2013'][linha]<=-0.5:
                arquivo.write('valor da correlacao:'+'\n')
                arquivo.write('tfg_1_2013:'+str(corr_tfg_2013['tfg_1_2013'][linha])+'  ')
                arquivo.write('tfg_2_2013:'+str(corr_tfg_2013['tfg_2_2013'][linha])+'\n')
                arquivo.write(str(tfg_2013[tfg_2013.keys()[linha]].describe()))
                arquivo.write('\n')
                arquivo.write('\n')

            elif corr_tfg_2014['tfg_1_2014'][linha]>=0.5 or corr_tfg_2014['tfg_2_2014'][linha]>=0.5:
                arquivo.write('valor da correlacao:'+'\n')
                arquivo.write('tfg_1_2014:'+str(corr_tfg_2014['tfg_1_2014'][linha])+'  ')
                arquivo.write('tfg_2_2014:'+str(corr_tfg_2014['tfg_2_2014'][linha])+'\n')
                arquivo.write(str(tfg_2014[tfg_2014.keys()[linha]].describe()))
                arquivo.write('\n')
                arquivo.write('\n')
            elif corr_tfg_2014['tfg_1_2014'][linha]<=-0.5 or corr_tfg_2014['tfg_2_2014'][linha]<=-0.5:
                arquivo.write('valor da correlacao:'+'\n')
                arquivo.write('tfg_1_2014:'+str(corr_tfg_2014['tfg_1_2014'][linha])+'  ')
                arquivo.write('tfg_2_2014:'+str(corr_tfg_2014['tfg_2_2014'][linha])+'\n')
                arquivo.write(str(tfg_2014[tfg_2014.keys()[linha]].describe()))
                arquivo.write('\n')
                arquivo.write('\n')